In [2]:
# Determining tail VaR of given portfolio
import numpy as np
from scipy.stats import norm, uniform
import matplotlib.pyplot as plt
import random
import math

%matplotlib inline

# General Share Information

In [13]:
S0 = np.array([[100],[95],[50]])
sigma = np.array([[0.15],[0.2],[0.3]])
cor_mat = np.array([[1, 0.2, 0.4],[0.2, 1, 0.8],[0.4, 0.8, 1]])

L = np.linalg.cholesky(cor_mat)
r = 0.1
T = 1

# Test S0 array

In [14]:
S0

array([[100],
       [ 95],
       [ 50]])

In [15]:
cor_mat

array([[1. , 0.2, 0.4],
       [0.2, 1. , 0.8],
       [0.4, 0.8, 1. ]])

In [16]:
L

array([[1.        , 0.        , 0.        ],
       [0.2       , 0.9797959 , 0.        ],
       [0.4       , 0.73484692, 0.54772256]])

# Caclulate our initial portfolio value

In [17]:
np.random.seed(0)
t_simulations = 1000
alpha = 0.05

Current portfolio value:

In [18]:
portval_current = np.sum(S0)
portval_current

245

Import the Terminal share price formula

In [19]:
def terminal_shareprice(S_0, r, sigma, Z, T):
    """Generates the terminal share price given some random normal values, Z"""
    a = r - sigma**2 / 2
    b = sigma * np.sqrt(T) * Z
    c = np.exp(a * T + b)
    result = S_0 * c
    return result

# Simulate future share values (assume no change in holding stocks)

In [20]:
Z = np.matmul(L, norm.rvs(size = [3, t_simulations]))
Z.shape

(3, 1000)

In [22]:
portval_future = np.sum(terminal_shareprice(S0, r, sigma, Z, T), axis=0)
portval_future.shape

(1000,)

In [23]:
portreturn = (portval_future - portval_current) / portval_current

In [25]:
portreturn = np.sort(portreturn)

In [27]:
mVaR_est = -portreturn[int(np.floor(alpha * t_simulations)) - 1]

In [28]:
mVaR_est

0.14318361044360806

# Using Historical Simulation to estimate VaR

In [29]:
random.seed(0)

In [30]:
import math
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import random

from scipy.stats import norm, uniform

In [31]:
S0 = 25
sigma = 0.1
mu = 0.1
T = 1
alpha = 0.01

np.random.seed(0)
dT = 1/365

In [38]:
def share_path(S_0, r, sigma, Z, dT):
    """Generates the terminal share price given some random normal values, Z"""
    a = (r - sigma**2/2) * dT
    b = sigma * np.sqrt(dT) * Z
    c = np.cumsum(a + b, 1)
    result = S_0 * np.exp(c)
    return result

In [39]:
Z_histdata = norm.rvs(size = [1, 10*365])
price_path = share_path(S0, mu, sigma, Z_histdata, dT)

In [40]:
price_path.shape

(1, 3650)

In [41]:
hist_lret = np.log(price_path[0, 1:] - np.log(price_path[0, 0:-1]))

In [42]:
mc_mu = np.mean(hist_lret) * 365
mc_sigma = np.std(hist_lret) * np.sqrt(365)

In [43]:
def terminal_shareprice_adj(S_0, mu, sigma, Z, T):
    a = mu * T
    b = sigma * np.sqrt(T) * Z
    c = np.exp(a + b)
    result = S_0 * c
    return result

In [44]:
alpha = 0.01
S0 = price_path[0, -1]
mportval_current = S0

mV_estimate = [None] * 50
histV_estimate = [None] * 50

In [45]:
for i in range(1,51):
    Z = norm.rvs(size = i * 1000)
    montportval_future = terminal_shareprice_adj(S0, mc_mu, mc_sigma, Z, T)
    hist_portret = [None] * i * 1000
    
    for j in range (1, i * 1000 + 1):
        rand_samp = uniform.rvs(size = 365) * (len(price_path[0]) - 1)
        rand_samp = [int(x) for x in rand_samp]
        share_returns = hist_lret[rand_samp]
        s_term = S0 * np.exp(np.sum(share_returns, axis = 0))
        hist_portret[j - i] = (s_term - mportval_current) / mportval_current
        
    montport_return = np.sort((montportval_future - mportval_current) / mportval_current)
    hist_portret = np.sort(hist_portret)
    
    mV_estimate[i - 1] = -montport_return[int(alpha * i * 1000) - 1]
    histV_estimate[i - 1] = -hist_portret[int(np.floor(alpha * i * 1000)) - 1]
    

/home/haininhhoang94/anaconda3/envs/pyds/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/home/haininhhoang94/anaconda3/envs/pyds/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [46]:
np.mean(histV_estimate)

-inf